# RAG система для оптимізації університетів з Gemini API

Цей notebook демонструє роботу RAG (Retrieval-Augmented Generation) системи для аналізу документів про покращення QS рейтингу університетів, використовуючи Gemini API та ChromaDB.


In [92]:
# =============================================================================
# ІМПОРТИ ТА ЗАЛЕЖНОСТІ
# =============================================================================

# Стандартні бібліотеки
import os
import logging
from pathlib import Path
from typing import Optional, List, Any, Dict, Tuple

# LangChain компоненти
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.language_models.llms import LLM
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_google_genai import ChatGoogleGenerativeAI

# Зовнішні API
import google.generativeai as genai
from dotenv import load_dotenv

# Налаштування логування
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Завантажуємо змінні середовища
load_dotenv()


True

In [ ]:
# =============================================================================
# КОНФІГУРАЦІЯ СИСТЕМИ
# =============================================================================

class RAGConfig:
    """Конфігурація для RAG системи"""
    
    # Налаштування API
    API_KEY = os.getenv("GOOGLE_API_KEY")
    GEMINI_MODEL = "gemini-2.0-flash"
    TEMPERATURE = 0.3
    MAX_OUTPUT_TOKENS = 2048
    
    # Налаштування документів
    DOCUMENT_FILES = [
        "docs/Покращення рейтингу QS українських університетів_ стратегії та інноваційні підходи.pdf",
        "docs/KPI ректора за 2024.pdf"
    ]
    
    # Налаштування чанків
    CHUNK_SIZE = 700
    CHUNK_OVERLAP = 50
    
    # Налаштування embeddings
    EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2" # глянути чи мультилінгуал
    VECTOR_DB_PATH = "./chroma_db"
    
    # Налаштування retriever
    RETRIEVER_K = 4
    RETRIEVER_SEARCH_TYPE = "mmr"  # MMR для кращої різноманітності
    RETRIEVER_FETCH_K = 20  # Отримуємо більше документів для фільтрації
    RETRIEVER_LAMBDA_MULT = 0.7  # Баланс релевантності та різноманітності
    
    # Налаштування промптів
    SYSTEM_ROLE = "експертний аналітичний радник ректора університету з досвідом роботи з QS рейтингами"

# Ініціалізація конфігурації
config = RAGConfig()

# Перевірка API ключа
if config.API_KEY:
    genai.configure(api_key=config.API_KEY)
    logger.info("✅ Gemini API налаштовано")
else:
    logger.warning("⚠️ GOOGLE_API_KEY не знайдено. Створіть .env файл з ключем.")

# Перевірка файлів
logger.info("📁 Перевірка файлів...")
valid_files = []
for file_path in config.DOCUMENT_FILES:
    if os.path.exists(file_path):
        logger.info(f"✅ {file_path}")
        valid_files.append(file_path)
    else:
        logger.warning(f"❌ {file_path} - файл не знайдено")

if not valid_files:
    raise FileNotFoundError("Не знайдено жодного валідного файлу для обробки")


2025-10-14 01:05:20,783 - INFO - ✅ Gemini API налаштовано
2025-10-14 01:05:20,785 - INFO - 📁 Перевірка файлів...
2025-10-14 01:05:20,786 - INFO - ✅ docs/Покращення рейтингу QS українських університетів_ стратегії та інноваційні підходи.pdf
2025-10-14 01:05:20,787 - INFO - ✅ docs/KPI ректора за 2024.pdf


In [94]:
# =============================================================================
# ФУНКЦІЇ ОБРОБКИ ДОКУМЕНТІВ
# =============================================================================

def load_documents(file_paths: List[str]) -> List[Any]:
    """
    Завантажує документи з PDF файлів
    
    Args:
        file_paths: Список шляхів до PDF файлів
        
    Returns:
        Список завантажених документів
        
    Raises:
        FileNotFoundError: Якщо не знайдено жодного валідного файлу
    """
    logger.info("📚 Завантаження документів...")
    docs_before_split = []
    
    for file_path in file_paths:
        if os.path.exists(file_path):
            try:
                loader = PyPDFLoader(file_path)
                docs = loader.load()
                docs_before_split.extend(docs)
                logger.info(f"✅ Завантажено {len(docs)} сторінок з {Path(file_path).name}")
            except Exception as e:
                logger.error(f"❌ Помилка завантаження {file_path}: {e}")
        else:
            logger.warning(f"⚠️ Пропускаємо {file_path} - файл не знайдено")
    
    if not docs_before_split:
        raise FileNotFoundError("Не вдалося завантажити жодного документа")
    
    logger.info(f"📄 Загалом завантажено {len(docs_before_split)} документів")
    if docs_before_split:
        logger.info(f"📝 Перший документ: {docs_before_split[0].page_content[:200]}...")
    
    return docs_before_split

def split_documents(documents: List[Any], chunk_size: int = 700, chunk_overlap: int = 50) -> List[Any]:
    """
    Розбиває документи на чанки
    
    Args:
        documents: Список документів для розбиття
        chunk_size: Розмір чанка
        chunk_overlap: Перекриття між чанками
        
    Returns:
        Список розбитих документів
    """
    logger.info("✂️ Розбиття на чанки...")
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    
    try:
        docs_after_split = text_splitter.split_documents(documents)
        logger.info(f"✅ Створено {len(docs_after_split)} чанків")
        
        if docs_after_split:
            logger.info(f"📝 Перший чанк: {docs_after_split[0].page_content[:300]}...")
            logger.info(f"📏 Довжина чанка: {len(docs_after_split[0].page_content)} символів")
        
        return docs_after_split
        
    except Exception as e:
        logger.error(f"❌ Помилка розбиття на чанки: {e}")
        raise

# Завантаження документів
docs_before_split = load_documents(valid_files)


2025-10-14 01:05:29,500 - INFO - 📚 Завантаження документів...
2025-10-14 01:05:31,055 - INFO - ✅ Завантажено 7 сторінок з Покращення рейтингу QS українських університетів_ стратегії та інноваційні підходи.pdf
2025-10-14 01:05:31,120 - INFO - ✅ Завантажено 1 сторінок з KPI ректора за 2024.pdf
2025-10-14 01:05:31,120 - INFO - 📄 Загалом завантажено 8 документів
2025-10-14 01:05:31,120 - INFO - 📝 Перший документ: Покращення  рейтингу  QS  українських  
університетів:
 
стратегії
 
та
 
інноваційні
 
підходи
 
Вступ  
Глобальна  конкуренція  у  сфері  вищої  освіти  спонукає  університети  боротися  за  вищі  м...


In [95]:
# Розбиття документів на чанки
docs_after_split = split_documents(
    docs_before_split, 
    chunk_size=config.CHUNK_SIZE, 
    chunk_overlap=config.CHUNK_OVERLAP
)


2025-10-14 01:05:39,105 - INFO - ✂️ Розбиття на чанки...
2025-10-14 01:05:39,139 - INFO - ✅ Створено 37 чанків
2025-10-14 01:05:39,139 - INFO - 📝 Перший чанк: Покращення  рейтингу  QS  українських  
університетів:
 
стратегії
 
та
 
інноваційні
 
підходи
 
Вступ  
Глобальна  конкуренція  у  сфері  вищої  освіти  спонукає  університети  боротися  за  вищі  місця  
у
 
міжнародних
 
рейтингах.
 
Високі
 
позиції
 
у
 
QS
 
World
 
University
 
Rankings
 
є
...
2025-10-14 01:05:39,139 - INFO - 📏 Довжина чанка: 696 символів


In [96]:
# =============================================================================
# ФУНКЦІЇ ВЕКТОРНОЇ БАЗИ ДАНИХ
# =============================================================================

def create_embeddings(model_name: str = "sentence-transformers/all-MiniLM-L6-v2") -> HuggingFaceEmbeddings:
    """
    Створює embeddings для документів
    
    Args:
        model_name: Назва моделі для embeddings
        
    Returns:
        Налаштований об'єкт embeddings
    """
    logger.info("🧠 Створення embeddings...")
    
    try:
        embeddings = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs={'device': 'cpu'},
            encode_kwargs={'normalize_embeddings': True}
        )
        logger.info("✅ Embeddings створено успішно")
        return embeddings
    except Exception as e:
        logger.error(f"❌ Помилка створення embeddings: {e}")
        raise

def create_vectorstore(documents: List[Any], embeddings: HuggingFaceEmbeddings, persist_directory: str) -> Chroma:
    """
    Створює векторну базу даних з документів
    
    Args:
        documents: Список документів для індексації
        embeddings: Об'єкт embeddings
        persist_directory: Шлях для збереження бази даних
        
    Returns:
        Налаштована векторна база даних
    """
    logger.info("🗄️ Створення векторної бази даних...")
    
    try:
        vectorstore = Chroma.from_documents(
            documents=documents,
            embedding=embeddings,
            persist_directory=persist_directory
        )
        logger.info("✅ Векторна база даних Chroma створена успішно")
        return vectorstore
    except Exception as e:
        logger.error(f"❌ Помилка створення векторної бази: {e}")
        raise

# Створення embeddings та векторної бази даних
embeddings = create_embeddings(config.EMBEDDING_MODEL)
vectorstore = create_vectorstore(docs_after_split, embeddings, config.VECTOR_DB_PATH)


2025-10-14 01:05:46,410 - INFO - 🧠 Створення embeddings...
2025-10-14 01:05:46,417 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2025-10-14 01:05:50,378 - INFO - ✅ Embeddings створено успішно
2025-10-14 01:05:50,388 - INFO - 🗄️ Створення векторної бази даних...
2025-10-14 01:05:51,638 - INFO - ✅ Векторна база даних Chroma створена успішно


In [ ]:
# =============================================================================
# ФУНКЦІЇ LLM ТА ПРОМПТІВ
# =============================================================================

def create_llm(model_name: str = "gemini-2.0-flash", temperature: float = 0.3, max_tokens: int = 2048) -> ChatGoogleGenerativeAI:
    """
    Створює LLM для генерації відповідей
    
    Args:
        model_name: Назва моделі
        temperature: Температура для генерації
        max_tokens: Максимальна кількість токенів
        
    Returns:
        Налаштований LLM
    """
    logger.info("🤖 Створення LLM...")
    
    try:
        llm = ChatGoogleGenerativeAI(
            model=model_name,
            temperature=temperature,
            max_output_tokens=max_tokens
        )
        logger.info("✅ Gemini LLM створено через офіційний інтегратор")
        return llm
    except Exception as e:
        logger.error(f"❌ Помилка створення LLM: {e}")
        raise

def create_prompt_template(system_role: str) -> PromptTemplate:
    """
    Створює шаблон промпту для RAG системи
    
    Args:
        system_role: Роль системи в промпті
        
    Returns:
        Налаштований шаблон промпту
    """
    logger.info("📝 Створення промпт шаблону...")
    
    prompt_template = f"""<|system|>
Ти — {system_role}.

Відповідай на питання на основі наданого контексту:

{{context}}

</s>
<|user|>
{{question}}
</s>
<|assistant|>

Відповідь (українською, конкретно та практично, СТИСЛО - максимум 2-3 абзаци):"""

    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template=prompt_template,
    )
    
    logger.info("✅ Промпт шаблон створено")
    return prompt

# Створення LLM та промптів
llm = create_llm(config.GEMINI_MODEL, config.TEMPERATURE, config.MAX_OUTPUT_TOKENS)
prompt = create_prompt_template(config.SYSTEM_ROLE)


2025-10-14 01:06:09,560 - INFO - 🤖 Створення LLM...


E0000 00:00:1760393169.673288  549869 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
2025-10-14 01:06:09,767 - INFO - ✅ Gemini LLM створено через офіційний інтегратор
2025-10-14 01:06:09,768 - INFO - 📝 Створення промпт шаблону...
2025-10-14 01:06:09,780 - INFO - ✅ Промпт шаблон створено


In [ ]:
# Створення промпт шаблону
prompt_template = """<|system|>
Ти — експертний аналітичний радник ректора університету з досвідом роботи з QS рейтингами.

Відповідай на питання на основі наданого контексту:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

Відповідь (українською, конкретно та практично, СТИСЛО - максимум 2-3 абзаци):"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

print("✅ Промпт шаблон створено")


✅ Промпт шаблон створено


In [99]:
# Створення LLM ланцюжка
llm_chain = prompt | llm | StrOutputParser()
print("✅ LLM ланцюжок створено")


✅ LLM ланцюжок створено


In [100]:
# =============================================================================
# ФУНКЦІЇ RAG СИСТЕМИ
# =============================================================================

def create_advanced_retriever(vectorstore: Chroma, k: int = None, search_type: str = None, 
                             fetch_k: int = None, lambda_mult: float = None) -> Any:
    """
    Створює покращений retriever з різними стратегіями пошуку
    
    Args:
        vectorstore: Векторна база даних
        k: Кількість документів для повернення
        search_type: Тип пошуку
        fetch_k: Кількість документів для отримання перед фільтрацією
        lambda_mult: Параметр балансу для MMR
        
    Returns:
        Налаштований retriever
    """
    logger.info("🔍 Створення покращеного retriever...")
    
    # Використовуємо значення з конфігурації якщо не вказано
    k = k or config.RETRIEVER_K
    search_type = search_type or config.RETRIEVER_SEARCH_TYPE
    fetch_k = fetch_k or config.RETRIEVER_FETCH_K
    lambda_mult = lambda_mult or config.RETRIEVER_LAMBDA_MULT
    
    try:
        # Створюємо retriever з параметрами
        search_kwargs = {"k": k}
        
        # Додаємо специфічні параметри для MMR
        if search_type == "mmr":
            search_kwargs.update({
                "fetch_k": fetch_k,
                "lambda_mult": lambda_mult
            })
        elif search_type == "similarity" and fetch_k:
            search_kwargs["fetch_k"] = fetch_k
            
        retriever = vectorstore.as_retriever(
            search_type=search_type,
            search_kwargs=search_kwargs
        )
        
        logger.info(f"✅ Покращений retriever створено (тип: {search_type})")
        return retriever
    except Exception as e:
        logger.error(f"❌ Помилка створення retriever: {e}")
        raise

def format_docs_with_sources(docs: List[Any]) -> str:
    """
    Форматує документи з додаванням інформації про джерела
    
    Args:
        docs: Список документів
        
    Returns:
        Відформатований текст з джерелами
    """
    formatted_docs = []
    seen_content = set()  # Для уникнення дублікатів
    
    for i, doc in enumerate(docs, 1):
        # Перевіряємо на дублікати
        content_hash = hash(doc.page_content[:100])  # Використовуємо перші 100 символів
        if content_hash in seen_content:
            continue
        seen_content.add(content_hash)
        
        source = doc.metadata.get('source', 'Невідомо')
        page = doc.metadata.get('page', 'Невідомо')
        
        formatted_doc = f"[Джерело {i}] {doc.page_content}"
        if source != 'Невідомо':
            filename = source.split('/')[-1] if '/' in source else source
            formatted_doc = f"[{filename}, сторінка {page}] {doc.page_content}"
        
        formatted_docs.append(formatted_doc)
    
    return "\n\n".join(formatted_docs)

def create_rag_chain(vectorstore: Chroma, llm: Any, prompt: PromptTemplate) -> Any:
    """
    Створює RAG ланцюжок з покращеним retriever
    
    Args:
        vectorstore: Векторна база даних
        llm: Модель мови
        prompt: Шаблон промпту
        
    Returns:
        Налаштований RAG ланцюжок
    """
    logger.info("🔗 Створення RAG ланцюжка...")
    
    try:
        # Створюємо покращений retriever з параметрами з конфігурації
        retriever = create_advanced_retriever(vectorstore)
        
        # Створюємо ланцюжок
        llm_chain = prompt | llm | StrOutputParser()
        
        rag_chain = (
            {"context": retriever | format_docs_with_sources, "question": RunnablePassthrough()}
            | llm_chain
        )
        
        logger.info("✅ RAG ланцюжок створено")
        return rag_chain, retriever
        
    except Exception as e:
        logger.error(f"❌ Помилка створення RAG ланцюжка: {e}")
        raise

# Створення RAG системи
rag_chain, retriever = create_rag_chain(vectorstore, llm, prompt)


2025-10-14 01:07:16,615 - INFO - 🔗 Створення RAG ланцюжка...
2025-10-14 01:07:16,617 - INFO - 🔍 Створення покращеного retriever...
2025-10-14 01:07:16,621 - INFO - ✅ Покращений retriever створено (тип: mmr)
2025-10-14 01:07:16,625 - INFO - ✅ RAG ланцюжок створено


In [101]:
# =============================================================================
# ФУНКЦІЇ ТЕСТУВАННЯ ТА ВІДОБРАЖЕННЯ
# =============================================================================

def test_retriever_diversity(retriever: Any, question: str, k: int = 4) -> List[Any]:
    """
    Тестує різноманітність результатів retriever
    
    Args:
        retriever: Retriever для тестування
        question: Питання для пошуку
        k: Кількість документів
        
    Returns:
        Список знайдених документів
    """
    logger.info("🔍 Тестування різноманітності retriever...")
    
    try:
        docs = retriever.invoke(question)
        
        # Аналізуємо різноманітність
        unique_sources = set()
        unique_pages = set()
        content_hashes = set()
        
        for doc in docs:
            source = doc.metadata.get('source', 'Невідомо')
            page = doc.metadata.get('page', 'Невідомо')
            content_hash = hash(doc.page_content[:100])
            
            unique_sources.add(source)
            unique_pages.add(page)
            content_hashes.add(content_hash)
        
        logger.info(f"📊 Статистика різноманітності:")
        logger.info(f"   - Унікальних джерел: {len(unique_sources)}")
        logger.info(f"   - Унікальних сторінок: {len(unique_pages)}")
        logger.info(f"   - Унікальних контентів: {len(content_hashes)}")
        
        return docs
        
    except Exception as e:
        logger.error(f"❌ Помилка тестування retriever: {e}")
        raise

def display_retrieved_docs(docs: List[Any], question: str) -> None:
    """
    Відображає знайдені документи з детальною інформацією
    
    Args:
        docs: Список документів
        question: Питання
    """
    print(f"\n❓ Питання: {question}")
    print("=" * 80)
    
    print(f"\n📚 Знайдено {len(docs)} документів:")
    
    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get('source', 'Невідомо')
        page = doc.metadata.get('page', 'Невідомо')
        
        filename = source.split('/')[-1] if source != 'Невідомо' else source
        
        print(f"\n📄 Документ {i}:")
        print(f"   📁 Файл: {filename}")
        print(f"   📄 Сторінка: {page}")
        print(f"   📝 Зміст: {doc.page_content[:200]}...")
        print(f"   📏 Довжина: {len(doc.page_content)} символів")

def test_rag_system(rag_chain: Any, retriever: Any, question: str) -> str:
    """
    Тестує RAG систему з детальним аналізом
    
    Args:
        rag_chain: RAG ланцюжок
        retriever: Retriever
        question: Питання
        
    Returns:
        Відповідь системи
    """
    logger.info("🤖 Тестування RAG системи...")
    
    try:
        # Тестуємо різноманітність retriever
        docs = test_retriever_diversity(retriever, question)
        
        # Відображаємо знайдені документи
        display_retrieved_docs(docs, question)
        
        print("\n" + "=" * 80)
        
        # Генеруємо відповідь
        print("\n💡 Відповідь RAG системи:")
        result = rag_chain.invoke(question)
        print(result)
        
        return result
        
    except Exception as e:
        logger.error(f"❌ Помилка тестування RAG: {e}")
        raise

# Тестування системи
question = "Як покращити QS рейтинг університету?"
result = test_rag_system(rag_chain, retriever, question)

2025-10-14 01:08:07,457 - INFO - 🤖 Тестування RAG системи...
2025-10-14 01:08:07,459 - INFO - 🔍 Тестування різноманітності retriever...
2025-10-14 01:08:07,740 - INFO - 📊 Статистика різноманітності:
2025-10-14 01:08:07,741 - INFO -    - Унікальних джерел: 1
2025-10-14 01:08:07,741 - INFO -    - Унікальних сторінок: 3
2025-10-14 01:08:07,742 - INFO -    - Унікальних контентів: 3



❓ Питання: Як покращити QS рейтинг університету?

📚 Знайдено 4 документів:

📄 Документ 1:
   📁 Файл: Покращення рейтингу QS українських університетів_ стратегії та інноваційні підходи.pdf
   📄 Сторінка: 4
   📝 Зміст: .
 
Тому
 
українським
 
політикам
 
варто
 
закладати
 
у
 
бюджет
 
пріоритетне
 
фінансування
 
науки
 
й
 
освіти,
 
стимулювати
 
приватні
 
інвестиції...
   📏 Довжина: 156 символів

📄 Документ 2:
   📁 Файл: Покращення рейтингу QS українських університетів_ стратегії та інноваційні підходи.pdf
   📄 Сторінка: 4
   📝 Зміст: .
 
Тому
 
українським
 
політикам
 
варто
 
закладати
 
у
 
бюджет
 
пріоритетне
 
фінансування
 
науки
 
й
 
освіти,
 
стимулювати
 
приватні
 
інвестиції...
   📏 Довжина: 156 символів

📄 Документ 3:
   📁 Файл: Покращення рейтингу QS українських університетів_ стратегії та інноваційні підходи.pdf
   📄 Сторінка: 2
   📝 Зміст: для
 
іноземців
 
і
 
створенню
 
спільних
 
освітніх
 
проєктів
 
із
 
західними
 
партнерами....
   📏 Довжина: 95 символів

In [88]:
# Тестування системи з виведенням джерел
print("🤖 Тестування RAG системи...")

# Тестове питання
question = "Як покращити QS рейтинг університету?"

print(f"\n❓ Питання: {question}")
print("=" * 80)

# Отримуємо релевантні документи
print("\n📚 Релевантні джерела:")
relevant_docs = retriever.invoke(question)
for i, doc in enumerate(relevant_docs, 1):
    source = doc.metadata.get('source', 'Невідомо')
    page = doc.metadata.get('page', 'Невідомо')
    print(f"\n📄 Джерело {i}:")
    print(f"   📁 Файл: {source.split('/')[-1] if source != 'Невідомо' else source}")
    print(f"   📄 Сторінка: {page}")
    print(f"   📝 Зміст: {doc.page_content[:200]}...")

print("\n" + "=" * 80)

# Тест з RAG (з контекстом)
print("\n🔍 Тест з RAG (з релевантним контекстом):")
result_with_rag = rag_chain.invoke(question)
print(result_with_rag)


🤖 Тестування RAG системи...

❓ Питання: Як покращити QS рейтинг університету?

📚 Релевантні джерела:

📄 Джерело 1:
   📁 Файл: Покращення рейтингу QS українських університетів_ стратегії та інноваційні підходи.pdf
   📄 Сторінка: 4
   📝 Зміст: .
 
Тому
 
українським
 
політикам
 
варто
 
закладати
 
у
 
бюджет
 
пріоритетне
 
фінансування
 
науки
 
й
 
освіти,
 
стимулювати
 
приватні
 
інвестиції...

📄 Джерело 2:
   📁 Файл: Покращення рейтингу QS українських університетів_ стратегії та інноваційні підходи.pdf
   📄 Сторінка: 4
   📝 Зміст: .
 
Тому
 
українським
 
політикам
 
варто
 
закладати
 
у
 
бюджет
 
пріоритетне
 
фінансування
 
науки
 
й
 
освіти,
 
стимулювати
 
приватні
 
інвестиції...

📄 Джерело 3:
   📁 Файл: Покращення рейтингу QS українських університетів_ стратегії та інноваційні підходи.pdf
   📄 Сторінка: 4
   📝 Зміст: .
 
Тому
 
українським
 
політикам
 
варто
 
закладати
 
у
 
бюджет
 
пріоритетне
 
фінансування
 
науки
 
й
 
освіти,
 
стимулювати
 
приватні
 
інвестиції

In [89]:
# Додаткові тестові питання з джерелами
test_questions = [
    "Що таке Citations per Faculty і як його збільшити?",
    "Які стратегії інтернаціоналізації університетів?",
    "Як підвищити академічну репутацію університету?"
]

for question in test_questions:
    print(f"\n❓ Питання: {question}")
    print("=" * 80)
    
    # Показуємо релевантні джерела
    print("\n📚 Релевантні джерела:")
    relevant_docs = retriever.invoke(question)
    for i, doc in enumerate(relevant_docs, 1):
        source = doc.metadata.get('source', 'Невідомо')
        page = doc.metadata.get('page', 'Невідомо')
        print(f"\n📄 Джерело {i}:")
        print(f"   📁 Файл: {source.split('/')[-1] if source != 'Невідомо' else source}")
        print(f"   📄 Сторінка: {page}")
        print(f"   📝 Зміст: {doc.page_content[:150]}...")
    
    print("\n" + "-" * 40)
    
    # Генеруємо відповідь
    try:
        result = rag_chain.invoke(question)
        print(f"\n💡 Відповідь:\n{result}")
    except Exception as e:
        print(f"❌ Помилка: {e}")
    
    print("\n" + "=" * 80)



❓ Питання: Що таке Citations per Faculty і як його збільшити?

📚 Релевантні джерела:

📄 Джерело 1:
   📁 Файл: Покращення рейтингу QS українських університетів_ стратегії та інноваційні підходи.pdf
   📄 Сторінка: 4
   📝 Зміст: .
 
Тому
 
українським
 
політикам
 
варто
 
закладати
 
у
 
бюджет
 
пріоритетне
 
фінансування
 
науки
 
й
 
освіти,
 
стимулювати
 
приватні
 
інве...

📄 Джерело 2:
   📁 Файл: Покращення рейтингу QS українських університетів_ стратегії та інноваційні підходи.pdf
   📄 Сторінка: 4
   📝 Зміст: .
 
Тому
 
українським
 
політикам
 
варто
 
закладати
 
у
 
бюджет
 
пріоритетне
 
фінансування
 
науки
 
й
 
освіти,
 
стимулювати
 
приватні
 
інве...

📄 Джерело 3:
   📁 Файл: Покращення рейтингу QS українських університетів_ стратегії та інноваційні підходи.pdf
   📄 Сторінка: 4
   📝 Зміст: .
 
Тому
 
українським
 
політикам
 
варто
 
закладати
 
у
 
бюджет
 
пріоритетне
 
фінансування
 
науки
 
й
 
освіти,
 
стимулювати
 
приватні
 
інве...

📄 Джерело 4:
   📁 Файл: Покр

In [ ]:
question = """QS Score рейтинг для ХПІ університету за 2025 рік рівний 11.17, за допомогою генетичного алгоритму було знайдено рішення що його можна покращити до 11.75 змбільшуючи Цитування та Інтернаціоналізацію на 1% 
1. Як можна покращити ці два показники?
2. Зважаючи на це рішення, які стратегії можна застосувати для покращення показників KPI ректора, які з них ти б міг запропонувати? (Зверни увагу на показники за 2024 рік)"""

In [106]:
relevant_docs = retriever.invoke(question)
for i, doc in enumerate(relevant_docs, 1):
    source = doc.metadata.get('source', 'Невідомо')
    page = doc.metadata.get('page', 'Невідомо')
    print(f"\n📄 Джерело {i}:")
    print(f"   📁 Файл: {source.split('/')[-1] if source != 'Невідомо' else source}")
    print(f"   📄 Сторінка: {page}")
    print(f"   📝 Зміст: {doc.page_content[:150]}...")

print("\n" + "-" * 40)

# Генеруємо відповідь
try:
    result = rag_chain.invoke(question)
    print(f"\n💡 Відповідь:\n{result}")
except Exception as e:
    print(f"❌ Помилка: {e}")


📄 Джерело 1:
   📁 Файл: Покращення рейтингу QS українських університетів_ стратегії та інноваційні підходи.pdf
   📄 Сторінка: 0
   📝 Зміст: і
 
сигналізує
 
про
 
рівень
 
підготовки
 
студентів
 
до
 
ринку
 
праці.
 
 ●  Співвідношення  викладачів  до  студентів  –  15%  .  Цей  показник...

📄 Джерело 2:
   📁 Файл: Покращення рейтингу QS українських університетів_ стратегії та інноваційні підходи.pdf
   📄 Сторінка: 0
   📝 Зміст: і
 
сигналізує
 
про
 
рівень
 
підготовки
 
студентів
 
до
 
ринку
 
праці.
 
 ●  Співвідношення  викладачів  до  студентів  –  15%  .  Цей  показник...

📄 Джерело 3:
   📁 Файл: Покращення рейтингу QS українських університетів_ стратегії та інноваційні підходи.pdf
   📄 Сторінка: 3
   📝 Зміст: англійською).
 
У
 
Близькосхідному
 
регіоні
 
(MENA)
 
нещодавно
 
теж
 
побачили
 
плоди
 
стратегічного
 
планування:
 
кількість
 
університетів
...

📄 Джерело 4:
   📁 Файл: Покращення рейтингу QS українських університетів_ стратегії та інноваційні підходи.pdf
   